In [ ]:
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from pylab import *
import pandas as pds
from sklearn import manifold
%matplotlib inline

# Importation des données

Les données étudiées dans ce TP provienne de la plateforme de données [Dataverse](https://dataverse.harvard.edu/dataset.xhtml?persistentId=hdl:1902.1/12379), elles contiennent des indices de similarité entre les votes des pays à l'ONU. 

- Télécharger les données de vote à l'ONU enregistrées dans le fichier [AffinitesVotesONU.txt](http://www.lsta.upmc.fr/ADDA/DataBertrand/AffinitesVotesONU.txt) (80 Mb) ainsi que le fichier [StateList.csv](http://www.lsta.upmc.fr/ADDA/DataBertrand/StateList.csv).
- Importer les données dans Python à l'aide la libraire Pandas.

Le fichier AffinitesVotesONU.txt est organisé comme indiqué ci-dessous :
- Une ligne renseigne la similarité des votes pour deux pays pendant une année.
- Les états sont codés par les champs "statea" et "stateb". 
- Le champ  "year" donne l'année considérée pour la comparaison des votes.
- Plusieurs indices de comparaison des votes sont fournis, dans la suite on s'intéresse à l'indice de similarité sur 3 niveaux  de votes (oui / non / abstension) :  "agree3un".

Afficher les premières lignes du dataframe VotingFull.

Le fichier StateList.csv est organisé comme indiqué ci-dessous :
- Une ligne correspond à un pays.
- Le champ "StateAbb" nous donne les abréviations (utile pour les représentations graphiques)
- Le champ "state" est le même code que celui utilisé dans VotingFull
- Le champ "CodeTP" est simplement le numéro de ligne
- Les champs "OPEP" et "UE" renseignent l'entrée dans l'OPEP ou dans l'UE (0 sinon).
Afficher les premières lignes du dataframe StateList.

In [ ]:
path_affinity= "/Users/bmichel/Dropbox/Enseignements/ADD-M2Stat/Tps-Notebooks/Datasets/Voting-ONU/AffinitesVotesONU.txt"
VotingFull = pds.read_csv(path_affinity,delim_whitespace=True,header=0)
path_affinity= "/Users/bmichel/Dropbox/Enseignements/ADD-M2Stat/Tps-Notebooks/Datasets/Voting-ONU/StateList.csv"
ListCountry = pds.read_csv(path_affinity,header=0)

Affichage des premières lignes de VotingFull et ListCountry

In [ ]:
VotingFull.head()

In [ ]:
ListCountry.head()

# Extraction des similarités de vote pour une periode donnée

In [ ]:
# periode étudiée :
annee_debut = 2000
annee_fin  = 2015   

# indice max des indices "state" (.values : syntaxe numpy )
maxstate = max(ListCountry.state.values)

# indice max dans CodeTP
maxCodeTP = max(ListCountry.CodeTP.values)

Le vecteur Translate ci-dessous permet de passer du codage "state" au codage "CodeTP".

In [ ]:
Translate =   - np.ones(maxstate+1,dtype = int) #
for numline in range(maxCodeTP):
    Translate[ListCountry.state[numline]] =  ListCountry.CodeTP[numline]

Par exemple le pays de code statea = 40 a pour indice indice 3 (on part de 0) pour "CodeTP" :

In [ ]:
Translate[40]

### Création de la matrice de dissimilarité $Dissimil$ 

Les numéros de ligne et de colonnes de $Dissimil$ correspondent au code "CodeTP". On remplit la matrice de similarité en parcourant les lignes de VotingFull, en ne selectionant que l'année "annee_choice" (ce code peut prendre quelques secondes).

La dissimilarité d'une année t est définie par :  1 - agree3un(t).

On cumule les dissimilarités sur la periode étudiée : 
$$Dissimil (paysA,paysB) = \sum_{t= debut} ^{fin}  (1- agree3un(t)) $$ 

In [ ]:
# création de la matrice (de bonne dimension)
# que des 0 partout par défaut

Dissimil = np.zeros((maxCodeTP,maxCodeTP)) 

# boucle sur les lignes de VotingFull (prend quelques secondes ...)
for numline in range(VotingFull.shape[0]):
    if (VotingFull.year[numline] >= annee_debut)&(VotingFull.year[numline] <= annee_fin): 
        Dissimil[Translate[VotingFull.statea[numline]],Translate[VotingFull.stateb[numline]]] +=  1-VotingFull.agree3un[numline]
        Dissimil[Translate[VotingFull.stateb[numline]],Translate[VotingFull.statea[numline]]] +=  1- VotingFull.agree3un[numline]

In [ ]:
print(Dissimil[:,0])

Attention !  Certaines lignes n'ont pas été remplies. En effet tous les pays ne sont pas renseignés dans la matrice car certains pays n'existent pas ou plus à la date choisie. 
Par exemple : 

In [ ]:
Dissimil[50,]

On doit retirer ces pays de la matrice de similarité, ce sont les pays dont les lignes ou colonnes ont une somme égale à - maxCodeTP (car la matrice est de taille maxCodeTP x maxCodeTP).

In [ ]:
I = np.array([ i for i in range(maxCodeTP) if Dissimil.sum(1)[i] != 0])
print(I)

Extraction de la sous matrice de similarité :

In [ ]:
DissimilExtract   = Dissimil[I,:][:,I]  #  rq : Dissimil[I,I] ne fonctionne pas

Il faut aussi remplacer les -1  par des 0 sur la diagonale :

In [ ]:
n,p = DissimilExtract.shape

Extraction du sous tableau de  ListCountry :

In [ ]:
ListCountryExtract = ListCountry.iloc[I]
print(DissimilExtract[:,0])

# Classical MDS

**Exercice** : Ecrire une fonction Python pour l'algorithme CMDS :
- en entrée une matrice de dissimilarité et une dimension p
- en sortie : une configuration du nuage dans $\mathbb R^p$.
Ecrire cette fonction dans un fichier nommé ClassicalMDS.py, que vous pourrez utiliser dans le notebook en l'important comme suit (consultez par exemple ce [lien](https://fr.wikibooks.org/wiki/Apprendre_à_programmer_avec_Python/Fonctions_originales)) :

In [ ]:
from ClassicalMDS import *

In [ ]:
configCMDS = ClassicalMDS(Dissimil,3)
print(configCMDS)

In [ ]:
fig = plt.figure(figsize = (18,18))
ax = fig.add_subplot(111)
CMDSAxe0 =configCMDS[:,0]
CMDSAxe1 =configCMDS[:,1]
ax.scatter(CMDSAxe0,CMDSAxe1, marker = 'o', s =  1)
for label, x, y, OPEP,UE in zip(ListCountryExtract.StateAbb, CMDSAxe0,CMDSAxe1,ListCountryExtract.OPEP,ListCountryExtract.UE):
    ax.annotate(label, xy = (x+0.01, y))
    if OPEP > 0:
        ax.text(x+0.01,y,label, bbox={'facecolor':'red'})
    if UE > 0:
        ax.text(x+0.01,y,label, bbox={'facecolor':'blue'})        



#Metric MDS

In [ ]:
mds = manifold.MDS(n_components=2,dissimilarity="precomputed",n_init=10,max_iter=10000)
mdsVoting = mds.fit(DissimilExtract).embedding_
mds.fit(DissimilExtract).stress_  
# on ne gagne pas beaucoup par rapport à n = 20 par exemple

In [ ]:
%matplotlib inline
fig = plt.figure(figsize = (15,15))
ax = fig.add_subplot(111)
MDSAxe0 =mdsVoting[:,0]
MDSAxe1 =mdsVoting[:,1]
ax.scatter(MDSAxe0,MDSAxe1, marker = 'o', s =  1)
for label, x, y, OPEP,UE in zip(ListCountryExtract.StateAbb, MDSAxe0,MDSAxe1,ListCountryExtract.OPEP,ListCountryExtract.UE):
    ax.annotate(label, xy = (x+0.01, y))
    if OPEP > 0:
        ax.text(x+0.01,y,label, bbox={'facecolor':'red'})
    if UE > 0:
        ax.text(x+0.01,y,label, bbox={'facecolor':'blue'})        
    
        
        
